In [ ]:
import tensorflow as tf

# force gpu use
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

In [2]:
from modules.pinn import PINNModel, model1, model2, model3, optm1, de1, ic1
from modules.data_gen import DataGenerator, real_u1, get_data
from modules.accuracy import AccuracyCalc

In [ ]:
EPOCHS = 1000
koefs = [1]  # [1, 5, 100]
x = (0, 2)
y = (0, 2)

dg = DataGenerator(x, y, None, real_u1)
acc = AccuracyCalc(dg, (70, 70))

Compare models

In [ ]:
area_grid_size = (50, 50)
border_grid_size = (50, 50)
dataset = dg.inner_pairs(area_grid_size), dg.border_pairs(border_grid_size)

for modelf in [model1, model2, model3]:
    model = PINNModel(modelf(), optm1())
    model.fit(1, *dataset, EPOCHS)
    
    dg.update_predict(model.predict)
    acc.update_predicitons((70, 70))
    print(acc.mse(), acc.maxe())
    # print(model.loss(koef, *dataset))
    print()

Time past 10.117368221282959

0.008396425817106269 0.3557392358779907

Time past 8.61058521270752

0.014792604367491273 0.8388965129852295

Time past 8.207294702529907

0.011121878082387316 0.7775897979736328



In [9]:
# area_grid_size = (50, 50)
# border_grid_size = (50, 50)
# dataset = dg.inner_pairs(area_grid_size), dg.border_pairs(border_grid_size)

# for koef in koefs:
#     model = PINNModel(de1, ic1,
#         model2(), optm1(), "mae")
#     model.fit(koef, *dataset, EPOCHS)
    
#     dg.update_predict(model.predict)
#     acc.update_predicitons((70, 70))
#     print(acc.mse(), acc.maxe())
#     print(model.loss(koef, *dataset))
#     print()